In [1]:
from elasticsearch import Elasticsearch, helpers
from faker import Faker
from random import choice, randint
from hashlib import sha1

In [2]:
client = Elasticsearch("http://elasticsearch:9200")

In [3]:
client.indices.exists(index="test_data")

/tmp/ipykernel_298/3102827968.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.exists(index="test_data")


HeadApiResponse(False)

In [4]:
def flip_coin():
    return randint(0,1)

In [5]:
fake = Faker()

In [6]:
data = []
for i in range(10000):
    person = {}
    # Flipping a coin to determine whether or not to entering value for a field creates some null fields, common in document dbs.
    if flip_coin():
        person["name"] = fake.name()
    if flip_coin():
        person["age"] = randint(1,99)
    if flip_coin():
        person["address"] = fake.address()
    if flip_coin():
        person["phone"] = fake.phone_number()
    if person:
        data.append(person)

In [7]:
data[0]

{'name': 'Charles Navarro',
 'age': 46,
 'address': '9989 Brian Field\nWest Michael, FL 83529',
 'phone': '335-665-3761x662'}

In [8]:
def create_bulk_data(data, index_name):
    for d in data:
        yield {
            "_index":index_name,
            "_type":"_doc",
            "_id":sha1(str(d).encode("utf8")).hexdigest(),
            "_source":d
        }

In [9]:
bulk_data = create_bulk_data(data, "test_data")

In [10]:
res = helpers.bulk(client,bulk_data,stats_only=False, raise_on_error=False)

/tmp/ipykernel_298/3845731093.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = helpers.bulk(client,bulk_data,stats_only=False, raise_on_error=False)
/tmp/ipykernel_298/3845731093.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  res = helpers.bulk(client,bulk_data,stats_only=False, raise_on_error=False)


In [11]:
res

(9359, [])